In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
import trafficDataUtility as util

In [3]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [4]:
dataset = {}
dataset = util.getDatasetFromFile(path+"sectionInfo.txt")

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [5]:
filename = "PeriodTotalAverage.txt"
util.updateDatasetFromFile(path+filename, dataset)

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [6]:
classes = 10

In [7]:
#filename = "input20_all10_2.csv"
filename = "input10_all10.csv"
#filename = "input100_rspeed.csv"
#filename = "input" + str(classes) +"_all.csv"
#filename = "input10_2min.csv"
data = util.loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_all10.csv


In [8]:
#selectedList = dataset.keys()
selectedList = ['Section697','Section754','Section1212','Section852']
#print(dataset['Section697'])

In [9]:
def getFeedXY_norm(Xa,Ya,k,count,z):
    Yn = Ya[k:count]/z
    Yn = np.round(Yn)
    Xn = Xa[:count-k]
    for i in range(1,k):
        Xn = np.c_[Xn, Xa[i:count-k+i]]
    
    m = int((count-k)*0.7)
    Xt = Xn[0:m]
    Xval = Xn[m:count-k]
    Yt = Yn[0:m]
    Yval = Yn[m:count-k]

    return Xt,Xval,Yt,Yval

In [10]:
def tf_model_(timeLen,classes):
    
    X = tf.placeholder(tf.float32, [None, input_size])
    X_img = tf.reshape(X, [-1, 5, timeLen, 1])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([3,3,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)

    L3 = tf.reshape(L2, [-1, 3*10])

    W3 = tf.Variable(tf.random_normal([3*10, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L3,W3) + b3
    return X,Y,Y_one_hot,hypothesis


In [11]:
def tf_model(timeLen,classes,input_size):
    
    X = tf.placeholder(tf.float32, [None, input_size])
    X_img = tf.reshape(X, [-1, 5, timeLen, 1])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='VALID')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([2,2,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)

    L3 = tf.reshape(L2, [-1, 3*10])

    W3 = tf.Variable(tf.random_normal([30, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L3,W3) + b3
    return X,Y,Y_one_hot,hypothesis

In [12]:
def tf_model_3(timeLen,classes,input_size):
    
    X = tf.placeholder(tf.float32, [None, input_size])
    X_img = tf.reshape(X, [-1, 3, timeLen, 1])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='VALID')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([2,2,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)

    L3 = tf.reshape(L2, [-1, 10])

    W3 = tf.Variable(tf.random_normal([10, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L3,W3) + b3
    return X,Y,Y_one_hot,hypothesis

In [13]:
def cnnModel3(data,dataset,classes,selectedList):
    outfilename = "CNN_Result" + str(classes) + ".txt"
    f = open(outfilename,'w')

    num = 0
    for id in selectedList:
        if id not in data.columns:
            continue
        if dataset[id][5] < 144:
            continue
        if dataset[id][7] > 80:
            continue
        
        Xa = util.getX(id,data,dataset)
        if Xa is None:
            continue
        Ya = np.array(data[id]).reshape(-1,1)

        num += 1
        print(num, id, '-------------------------')
        timeLen = 3
        Xt,Xval,Yt,Yval = getFeedXY_norm(Xa,Ya,timeLen,1440,10)

        input_size = 5*timeLen
        
        X,Y,Y_one_hot,hypothesis = tf_model(timeLen,classes,input_size)    
    
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        prediction = tf.argmax(hypothesis, 1)
        correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for epoch in range(100):
            c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
        trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
        valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
        print('Training:', trainAccuracy)
        print('Validation:', valAccuracy)

        print(sess.run(prediction, feed_dict={X:Xval,Y:Yval}))
        
        log = id
        log += ',' + dataset[id][0]
        log += ',' + dataset[id][1]
        log += ',' + str(dataset[id][5])
        log += ',' + str(dataset[id][7])
        log += ',' + str(trainAccuracy)
        log += ',' + str(valAccuracy)
        log += '\n'
        f.write(log)
    f.close()

In [14]:
def cnnModel_3(data,dataset,classes,selectedList):
    outfilename = "CNN_Result" + str(classes) + ".txt"
    f = open(outfilename,'w')

    num = 0
    for id in selectedList:
        if id not in data.columns:
            continue
        if dataset[id][5] < 144:
            continue
        if dataset[id][7] > 80:
            continue
        
        Xa = util.getX_3(id,data,dataset)
        if Xa is None:
            continue
        Ya = np.array(data[id]).reshape(-1,1)

        num += 1
        print(num, id, '-------------------------')
        timeLen = 3
        Xt,Xval,Yt,Yval = getFeedXY_norm(Xa,Ya,timeLen,1440,10)

        input_size = 3*timeLen
        
        X,Y,Y_one_hot,hypothesis = tf_model_3(timeLen,classes,input_size)    
    
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        prediction = tf.argmax(hypothesis, 1)
        correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for epoch in range(100):
            c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
        trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
        valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
        print('Training:', trainAccuracy)
        print('Validation:', valAccuracy)

        print(sess.run(prediction, feed_dict={X:Xval,Y:Yval}))
        
        log = id
        log += ',' + dataset[id][0]
        log += ',' + dataset[id][1]
        log += ',' + str(dataset[id][5])
        log += ',' + str(dataset[id][7])
        log += ',' + str(trainAccuracy)
        log += ',' + str(valAccuracy)
        log += '\n'
        f.write(log)
    f.close()

In [15]:
#cnnModel2(data,dataset,classes,selectedList)
#cnnModel3(data,dataset,classes,selectedList)
cnnModel_3(data,dataset,classes,selectedList)


1 Section697 -------------------------
Training: 0.764179
Validation: 0.789352
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
2 Section754 -------------------------
Train